Using MS-COCO data set

In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
import tensorflow as tf
import numpy as np
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Concatenate
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import pickle
from torchvision import transforms
from torch.utils.data import DataLoader

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = "Downloads/train2014"
val_dataset = "Downloads/val2014"

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

2025-05-13 14:00:16.642457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747137616.661681  792746 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747137616.668193  792746 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747137616.692569  792746 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747137616.692585  792746 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747137616.692588  792746 computation_placer.cc:177] computation placer alr

Feature extraction

In [2]:
base_model = VGG16(weights='imagenet', include_top=False)
print(base_model.summary())

# Output shape will be (batch_size, 7, 7, 512) => flatten to (batch_size, 25088)


I0000 00:00:1747137675.217238  792746 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1747137675.218097  792746 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9005 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:20:00.0, compute capability: 7.5


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast (Cast)                     │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

None


In [3]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # shape (1, 224, 224, 3)
    img_array = preprocess_input(img_array)
    return img_array


Extract and save features

In [4]:
def extract_features_from_folder(folder_path, save_path):
    features = {}
    for fname in tqdm(os.listdir(folder_path)):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, fname)
            img_array = preprocess_image(img_path)
            feature = base_model.predict(img_array)
            feature = feature.reshape(feature.shape[0], -1)  # shape (1, 25088)
            features[fname] = feature[0]  # Store as (25088,) vector

    # Save all extracted features
    with open(save_path, 'wb') as f:
        pickle.dump(features, f)

    print(f"Features saved to {save_path}")


In [5]:
extract_features_from_folder('./train2014', 'train2014_features.pkl')

  0%|          | 0/664 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1747137677.148979  793049 service.cc:152] XLA service 0x7f759c005150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747137677.149012  793049 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2025-05-13 14:01:17.164940: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747137677.342416  793049 cuda_dnn.cc:529] Loaded cuDNN version 90501


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


I0000 00:00:1747137681.340384  793049 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
  0%|          | 1/664 [00:04<50:21,  4.56s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


  0%|          | 2/664 [00:04<21:27,  1.94s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


100%|█████████▉| 663/664 [01:01<00:00, 11.72it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


100%|██████████| 664/664 [01:01<00:00, 10.83it/s]


Features saved to train2014_features.pkl


In [6]:
# IMAGE FEATURE EXTRACTION
cnn_model = Model(inputs=base_model.input, outputs=base_model.output)

In [7]:
# Parameters (adjust as needed)
max_length = 34
vocab_size = 10000
embedding_dim = 256
units = 256
beam_size = 3

# IMAGE + TEXT MODEL
def build_model():
    # Image model
    image_input = Input(shape=(25088,), name='image_input')
    image_embedding = Dense(units, activation='relu')(image_input)

    # Caption model
    caption_input = Input(shape=(max_length,), name='caption_input')
    caption_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(caption_input)
    caption_lstm = LSTM(units)(caption_embedding)

    # Combine
    merged = Concatenate()([image_embedding, caption_lstm])
    output = Dense(vocab_size, activation='softmax')(merged)

    model = Model(inputs=[image_input, caption_input], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    print(model.summary())
    return model

In [8]:
# BEAM SEARCH DECODER
def beam_search_decoder(photo_feature, tokenizer, model, beam_size=3):
    start = tokenizer.word_index['startseq']
    end = tokenizer.word_index['endseq']

    sequences = [[list(), 0.0]]
    for _ in range(max_length):
        all_candidates = []
        for seq, score in sequences:
            if len(seq) > 0 and seq[-1] == end:
                all_candidates.append((seq, score))
                continue
            padded = pad_sequences([seq], maxlen=max_length)
            y_pred = model.predict([photo_feature, padded], verbose=0)
            top_indices = np.argsort(y_pred[0])[-beam_size:]
            for idx in top_indices:
                candidate = seq + [idx]
                candidate_score = score - np.log(y_pred[0][idx] + 1e-7)  # avoid log(0)
                all_candidates.append((candidate, candidate_score))
        sequences = sorted(all_candidates, key=lambda tup: tup[1])[:beam_size]
    best_seq = sequences[0][0]
    return best_seq

In [9]:
# 2. Build model
model = build_model()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 25088)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ caption_input       │ (None, 34)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_1 (Cast)       │ (None, 25088)     │          0 │ image_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 34, 256)   │  2,560,000 │ caption_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  6,422,784 │ cast_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10000)     │  5,130,000 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,638,096 (55.84 MB)

 Trainable params: 14,638,096 (55.84 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
import json
from collections import defaultdict
import os

def build_captions_dict_from_coco(json_path, images_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Build mapping from image ID to file name
    id_to_filename = {img['id']: img['file_name'] for img in data['images']}

    # Build captions_dict
    captions_dict = defaultdict(list)
    for ann in data['annotations']:
        img_id = ann['image_id']
        caption = ann['caption'].lower().strip()
        filename = id_to_filename[img_id]
        full_path = os.path.join(images_dir, filename)
        captions_dict[full_path].append(f"startseq {caption} endseq")

    return dict(captions_dict)

train_captions = build_captions_dict_from_coco(
    json_path='annotations/captions_train2014.json',
    images_dir='train2014'
)

val_captions = build_captions_dict_from_coco(
    json_path='annotations/captions_val2014.json',
    images_dir='val2014'
)

# Save if needed
import pickle
with open('train_captions_dict.pkl', 'wb') as f:
    pickle.dump(train_captions, f)

# Example preview
sample = list(train_captions.items())[0]
print("Image:", sample[0])
print("Captions:", sample[1])


Image: train2014/COCO_train2014_000000318556.jpg
Captions: ['startseq a very clean and well decorated empty bathroom endseq', 'startseq a blue and white bathroom with butterfly themed wall tiles. endseq', 'startseq a bathroom with a border of butterflies and blue paint on the walls above it. endseq', 'startseq an angled view of a beautifully decorated bathroom. endseq', 'startseq a clock that blends in with the wall hangs in a bathroom. endseq']


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='<unk>')
all_captions = [cap for caps in train_captions.values() for cap in caps]
tokenizer.fit_on_texts(all_captions)

# Optional: Save for later
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert captions to sequences
sequences = tokenizer.texts_to_sequences(all_captions)
max_length = max(len(seq) for seq in sequences)  # You can fix to 34 if needed
print(f"Max caption length: {max_length}")


Max caption length: 52


In [13]:
import numpy as np

def create_sequences(tokenizer, captions_dict, features, max_length, vocab_size):
    X1, X2, y = [], [], []

    for img_path, captions in tqdm(captions_dict.items()):
        img_name = os.path.basename(img_path)
        if img_name not in features:
            continue
        feature = features[img_name]

        for caption in captions:
            seq = tokenizer.texts_to_sequences([caption])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]

                out_seq_onehot = np.zeros(vocab_size)
                out_seq_onehot[out_seq] = 1.0  # One-hot encoding

                X1.append(feature)          # shape (25088,)
                X2.append(in_seq)           # shape (max_length,)
                y.append(out_seq_onehot)    # shape (vocab_size,)

    return np.array(X1), np.array(X2), np.array(y)


# Load pre-extracted features
with open('train2014_features.pkl', 'rb') as f:
    features = pickle.load(f)


In [14]:
max_length = 34
X1, X2, y = create_sequences(tokenizer, train_captions, features, max_length, vocab_size)
print(X1.shape, X2.shape, y.shape)  # should be (num_samples, 25088), (num_samples, max_length), (num_samples, vocab_size)

# Train
model.fit([X1, X2], y, epochs=10, batch_size=64, verbose=1)

# Save trained model
model.save('image_captioning_model.h5')


100%|██████████| 40504/40504 [00:02<00:00, 18418.11it/s]


(38325, 25088) (38325, 34) (38325, 10000)
Epoch 1/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.1253 - loss: 13.5039
Epoch 2/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.1484 - loss: 13.7264
Epoch 3/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.1489 - loss: 13.7181
Epoch 4/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.1480 - loss: 13.7324
Epoch 5/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1485 - loss: 13.7248
Epoch 6/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1450 - loss: 13.7804
Epoch 7/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.1475 - loss: 13.7400
Epoch 8/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.1466 - loss: 13.7551
Epoch 9/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1503 - loss: 13.6953
Epoch 10/10
599/599 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1514 - loss: 13.6784


In [17]:
from tensorflow.keras.models import load_model

# Load model and tokenizer if needed
# model = load_model('image_captioning_model.h5')
# with open('tokenizer.pkl', 'rb') as f:
#     tokenizer = pickle.load(f)

def generate_caption(photo_feature, tokenizer, model, max_length=34):
    in_text = ['startseq']
    for _ in range(max_length):
        seq = tokenizer.texts_to_sequences([in_text])[0]
        seq = pad_sequences([seq], maxlen=max_length)
        yhat = model.predict([photo_feature, seq], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat, None)
        if word is None:
            break
        in_text.append(word)
        if word == 'endseq':
            break
    return ' '.join(in_text[1:-1])  # skip 'startseq' and 'endseq'

# Example: Use any test image feature
img_path = "val2014/COCO_val2014_000000000073.jpg"
img_array = preprocess_image(img_path)
img_feature = base_model.predict(img_array).reshape(1, 25088)
caption = generate_caption(img_feature, tokenizer, model)
print("Generated Caption:", caption)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Generated Caption: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a


In [18]:
from nltk.translate.bleu_score import sentence_bleu

# Example
reference = [cap.split() for cap in val_captions[img_path]]
candidate = caption.split()
bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {bleu_score}")


BLEU Score: 1.0003688322288243e-231


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [10]:
# 3. (Training loop should be added here with your preprocessed caption data)

# 4. Generate captions after training:
# tokenizer = ... (load your tokenizer)
# photo_feature = ... (load image feature)
# best_seq = beam_search_decoder(photo_feature.reshape(1, 25088), tokenizer, model)
# caption = ' '.join(tokenizer.index_word[i] for i in best_seq if i in tokenizer.index_word)